In [ ]:
import os
import numpy as np
from torch.utils.data import DataLoader
from init import seed_everything, dump_args, data_preprocess
from dataset import CharDataset
from classifier import Classifier

In [ ]:
args = {}
args["model"] = "LeNet"                      # ["LeNet", "ResNet"]
args["epoches"] = 50
args["batch_size"] = 64
args["learning_rate"] = 0.06
args["record_path"] = "./record"
args["save_path"] = "./save"                 # 完整路径是 f"./save/{args['model']}_best.ckpt"
args["raw_data_path"] = "./data/raw"
args["data_path"] = "./data/data.npz"
args["random_seed"] = 42
args["mode"] = "train_and_test"              # ["train", "test", "train_and_test"]

args["save_path"] = os.path.join(args["save_path"], f"{args['model']}_best.ckpt")
record_path = dump_args(args, args["record_path"], args["mode"])

In [ ]:
if not os.path.exists(args["data_path"]):
    data_preprocess(args["raw_data_path"], args["data_path"], shuffle=True)
data_npz = np.load(args["data_path"])

dataset_train = CharDataset(data_npz["x_train"], data_npz["y_train"])
dataloader_train = DataLoader(dataset_train, args["batch_size"], shuffle=True)

dataset_valid = CharDataset(data_npz["x_valid"], data_npz["y_valid"])
dataloader_valid = DataLoader(dataset_valid, args["batch_size"], shuffle=False)

In [ ]:
classifier = Classifier()
if os.path.exists(args["save_path"]):
    classifier.load_model(args["save_path"])
    print(classifier.best)